# Object Detection

In [1]:
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available.")


GPU available: NVIDIA H100 PCIe


In [3]:
from zipfile import ZipFile

# Path to your ZIP file
zip_file_path = '/user/bhanucha/final_data.zip'
# Destination directory where you want to extract the files
extraction_directory = '/user/bhanucha/input'

# Extract the ZIP file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

print("Extraction completed.")


Extraction completed.


In [4]:
import xml.etree.ElementTree as ET
import os
from PIL import Image

# Load class names and create a mapping to IDs
def load_class_names(class_file_path):
    with open(class_file_path, 'r') as file:
        class_names = file.read().strip().split('\n')
    return {name: i for i, name in enumerate(class_names)}

def convert_voc_to_yolo(voc_xml_file, class_mapping, img_width, img_height):
    tree = ET.parse(voc_xml_file)
    root = tree.getroot()
    yolo_format = []

    for member in root.findall('object'):
        classname = member.find('name').text
        class_id = class_mapping[classname]

        bndbox = member.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_format.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_format

def process_dataset(dataset_directory, class_file_path):
    class_mapping = load_class_names(class_file_path)

    for class_dir in os.listdir(dataset_directory):
        class_path = os.path.join(dataset_directory, class_dir)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                if file.endswith('.xml'):
                    img_file = os.path.splitext(file)[0] + '.jpg'
                    img_path = os.path.join(class_path, img_file)
                    xml_path = os.path.join(class_path, file)

                    # Use PIL to get image dimensions
                    with Image.open(img_path) as img:
                        img_width, img_height = img.size

                    yolo_annotations = convert_voc_to_yolo(xml_path, class_mapping, img_width, img_height)
                    yolo_annotation_text = "\n".join(yolo_annotations)

                    # Save YOLO annotations to a .txt file
                    txt_filename = os.path.splitext(xml_path)[0] + '.txt'
                    with open(txt_filename, 'w') as f:
                        f.write(yolo_annotation_text)

# Assuming your class file path and dataset directory are as follows:
class_file_path = '/user/bhanucha/Final_classes.txt'
dataset_directory = '/user/bhanucha/input/initial_data_annotated'
process_dataset(dataset_directory, class_file_path)






In [5]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import random
from sklearn.model_selection import train_test_split


dataset_dir = '/user/bhanucha/input/initial_data_annotated'
output_train = '/user/bhanucha/train.txt'
output_val = '/user/bhanucha/valid.txt'
output_test = '/user/bhanucha/test.txt'


train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1  

image_paths = []
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))


train_val_paths, test_paths = train_test_split(image_paths, test_size=test_ratio, random_state=42)
train_paths, val_paths = train_test_split(train_val_paths, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)


def write_paths(file_paths, output_file):
    with open(output_file, 'w') as f:
        for path in file_paths:
            f.write(path + '\n')


write_paths(train_paths, output_train)
write_paths(val_paths, output_val)
write_paths(test_paths, output_test)

print(f"Training images: {len(train_paths)}")
print(f"Validation images: {len(val_paths)}")
print(f"Test images: {len(test_paths)}")


Training images: 3988
Validation images: 499
Test images: 499


## YOLO V7

In [9]:
!ls

LICENSE.md   deploy	 inference	   scripts	    train_aux.py
README.md    detect.py	 models		   test.py	    utils
__pycache__  export.py	 paper		   tools	    yolov7.pt
cfg	     figure	 requirements.txt  traced_model.pt
data	     hubconf.py  runs		   train.py


In [8]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd /projects/academic/courses/cse676s24/bhanucha/yolov7/


Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197
Receiving objects: 100% (1197/1197), 74.23 MiB | 57.50 MiB/s, done.
Resolving deltas: 100% (520/520), done.
/projects/academic/courses/cse676s24/bhanucha/yolov7


In [15]:
!pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 62.2 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 3.5 MB/s s eta 0:00:01
     |████████████████████████████████| 294 kB 103.5 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
yaml_content = """
train: /user/bhanucha/train.txt
val: /user/bhanucha/valid.txt
test: /user/bhanucha/test.txt

nc: 100
names: [
  'all_purpose_flour', 'almonds', 'apple', 'apricot', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'basil',
  'basmati_rice', 'beans', 'beef', 'beets', 'bell_pepper', 'berries', 'biscuits', 'blackberries', 'black_pepper',
  'blueberries', 'bread', 'bread_crumbs', 'bread_flour', 'broccoli', 'brownie_mix', 'brown_rice', 'butter', 'cabbage',
  'cake', 'cardamom', 'carrot', 'cashews', 'cauliflower', 'celery', 'cereal', 'cheese', 'cherries', 'chicken',
  'chickpeas', 'chocolate', 'chocolate_chips', 'chocolate_syrup', 'cilantro', 'cinnamon', 'clove', 'cocoa_powder',
  'coconut', 'cookies', 'corn', 'cucumber', 'dates', 'eggplant', 'eggs', 'fish', 'garlic', 'ginger', 'grapes', 'honey',
  'jalapeno', 'kidney_beans', 'lemon', 'mango', 'marshmallows', 'milk', 'mint', 'muffins', 'mushroom', 'noodles',
  'nuts', 'oats', 'okra', 'olive', 'onion', 'orange', 'oreo_cookies', 'pasta', 'pear', 'pepper', 'pineapple',
  'pistachios', 'pork', 'potato', 'pumpkin', 'radishes', 'raisins', 'red_chilies', 'rice', 'rosemary', 'salmon', 'salt',
  'shrimp', 'spinach', 'strawberries', 'sugar', 'sweet_potato', 'tomato', 'vanilla_ice_cream', 'walnuts', 'watermelon',
  'yogurt'
]
"""

with open('/user/bhanucha/ingredients.yaml', 'w') as file:
    file.write(yaml_content.strip())


In [21]:
%cd /projects/academic/courses/cse676s24/bhanucha/yolov7/yolov7


/projects/academic/courses/cse676s24/bhanucha/yolov7/yolov7


In [17]:
!python train.py --batch-size 64 --img 640 640 --data /user/bhanucha/ingredients.yaml --cfg cfg/training/yolov7.yaml --weights 'yolov7.pt' --device 0 --epochs 100 --workers 4

YOLOR 🚀 v0.1-128-ga207844 torch 1.13.1 CUDA:0 (NVIDIA H100 PCIe, 81230.375MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7.yaml', data='/user/bhanucha/ingredients.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=100, batch_size=64, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp', total_batch_size=64)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2024-04-29 19:49:41.712193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary i

In [19]:
!python test.py --weights /projects/academic/courses/cse676s24/bhanucha/yolov7/yolov7/runs/train/exp/weights/best.pt --data /user/bhanucha/ingredients.yaml --img 640 --iou-thres 0.5 --batch-size 64 --task test


Namespace(weights=['/projects/academic/courses/cse676s24/bhanucha/yolov7/yolov7/runs/train/exp/weights/best.pt'], data='/user/bhanucha/ingredients.yaml', batch_size=64, img_size=640, conf_thres=0.001, iou_thres=0.5, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-128-ga207844 torch 1.13.1 CUDA:0 (NVIDIA H100 PCIe, 81230.375MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/pytorch/1.13.1-CUDA-11.8.0/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /var/tmp/aebruno2/easybuild/build/PyTorch/1.13.1/foss-2021b-CUDA

## YOLO V9 Version1

In [10]:

%cd /projects/academic/courses/cse676s24/bhanucha/


/projects/academic/courses/cse676s24/bhanucha


In [3]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd /projects/academic/courses/cse676s24/bhanucha/yolov9/

Cloning into 'yolov9'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 668 (delta 222), reused 199 (delta 199), pack-reused 378
Receiving objects: 100% (668/668), 3.22 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (269/269), done.
/projects/academic/courses/cse676s24/bhanucha/yolov9


In [4]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached scipy-1.13.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.2 which is incompatible.
You should consider upgrading via the '/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/Compiler/gcccore/11.2.0/python/3.9.6/bin/python3.9 -m pip install --upgrade pip' command.


In [25]:
%cd /projects/academic/courses/cse676s24/bhanucha/yolov9


/projects/academic/courses/cse676s24/bhanucha/yolov9


In [6]:
!python train_dual.py --batch-size 32 --img 640 --data /user/bhanucha/ingredients.yaml --cfg models/detect/yolov9-c.yaml --weights '' --hyp hyp.scratch-high.yaml --device 0 --epochs 100 --workers 4

train_dual: weights=, cfg=models/detect/yolov9-c.yaml, data=/user/bhanucha/ingredients.yaml, hyp=hyp.scratch-high.yaml, epochs=100, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO 🚀 v0.1-89-g93f1a28 Python-3.9.6 torch-1.13.1 CUDA:0 (NVIDIA A100-PCIE-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, obj=0.7, obj_pw=1.0, dfl=1.5, iou_t=0.2, anchor_t=5.0, fl_

In [5]:
!python val_dual.py --weights /projects/academic/courses/cse676s24/bhanucha/yolov9/runs/train/exp/weights/best.pt --data /user/bhanucha/ingredients.yaml --img 640 --iou-thres 0.5 --batch-size 32 --save-json --name yolov9_c_c_640_val

val_dual: data=/user/bhanucha/ingredients.yaml, weights=['/projects/academic/courses/cse676s24/bhanucha/yolov9/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=yolov9_c_c_640_val, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-89-g93f1a28 Python-3.9.6 torch-1.13.1 CUDA:0 (NVIDIA A100-PCIE-40GB, 40514MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50926968 parameters, 0 gradients, 237.9 GFLOPs
val: Scanning /user/bhanucha/valid.cache... 499 images, 132 backgrounds, 0 corru
                 Class     Images  Instances          P          R      mAP50   
                   all        499        367       0.44      0.371      0.404      0.359
Speed: 0.1ms pre-process, 6.2ms inference, 0.9ms NMS per image at shape (32, 3, 640, 640)

Evaluating pyc

### V9_Version_2

In [29]:
!python train_dual.py --batch-size 16 --img 320 --data /user/bhanucha/ingredients.yaml --cfg models/detect/yolov9-c.yaml --weights '' --hyp hyp.scratch-high.yaml --device 0 --epochs 300 --workers 4

train_dual: weights=, cfg=models/detect/yolov9-c.yaml, data=/user/bhanucha/ingredients.yaml, hyp=hyp.scratch-high.yaml, epochs=300, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO 🚀 v0.1-89-g93f1a28 Python-3.9.6 torch-1.13.1 CUDA:0 (NVIDIA H100 PCIe, 81230MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, obj=0.7, obj_pw=1.0, dfl=1.5, iou_t=0.2, anchor_t=5.0, fl_gamma

In [30]:
!python val_dual.py --weights /projects/academic/courses/cse676s24/bhanucha/yolov9/runs/train/exp6/weights/best.pt --data /user/bhanucha/ingredients.yaml --img 640 --iou-thres 0.5 --batch-size 32 --save-json --name yolov9_c_c_640_val

val_dual: data=/user/bhanucha/ingredients.yaml, weights=['/projects/academic/courses/cse676s24/bhanucha/yolov9/runs/train/exp6/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=yolov9_c_c_640_val, exist_ok=False, half=False, dnn=False, min_items=0
YOLO 🚀 v0.1-89-g93f1a28 Python-3.9.6 torch-1.13.1 CUDA:0 (NVIDIA H100 PCIe, 81230MiB)

Fusing layers... 
yolov9-c summary: 604 layers, 50926968 parameters, 0 gradients, 237.9 GFLOPs
val: Scanning /user/bhanucha/valid.cache... 499 images, 132 backgrounds, 0 corru
                 Class     Images  Instances          P          R      mAP50   
                   all        499        367      0.267      0.289      0.278      0.186
Speed: 0.1ms pre-process, 4.4ms inference, 0.6ms NMS per image at shape (32, 3, 640, 640)

Evaluating pycocot